# 土炮股票分析系統

這個範例只是將上課所講過的所有例子串在一起，提供一個簡單的概念，讓大家知道一個簡單的分析系統大概可以怎麼做。

例子裡面有很多不足的地方，請自行思考如何修改。

這個範例主要做的事是下方方塊圖的藍色區塊部分。

In [ ]:
# 載入需要的模組
import ffn
import pandas as pd
import pandas_datareader.data as web

from datetime import datetime
import numpy as np

## 前面定義好的一些函式

In [ ]:
# 取得公司資料
def get_companies(ex = "NASDAQ"):
    template = "http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange={}&render=download"
    url = template.format(ex)
    return pd.read_csv(url)

In [ ]:
# 計算波動率
def volatility(symbol, startdate):
    df = web.DataReader(symbol, 'google', startdate)
    dailyRet = df['Close'].pct_change()
    return dailyRet.std()

In [ ]:
# 計算 MaxDD
def DrawDownAnalysis(cumRet):
    dd_series = ffn.core.to_drawdown_series(cumRet)
    dd_details = ffn.core.drawdown_details(dd_series)
    return dd_details['drawdown'].min(), dd_details['days'].max()

## 抓取公司資料

In [ ]:
data = get_companies()

In [ ]:
data.head()

In [ ]:
# 看一下有幾筆資料
len(data.index)

In [ ]:
# 也可以這樣看
data.shape

In [ ]:
# 作為範例取出前十筆來做就好了，因為太多筆沒辦法在課堂上示範...
# 另外連續頻繁抓資料，會被 Yahoo Finance 視為攻擊，之後會抓不到資料。
companylist = data['Symbol'][0:10].tolist()

## 波動率選股

我們通常只抓一次股市資料，然後存到資料庫裏面去，之後就從自己的資料庫裏面撈數據出來分析。

這個地方因為為了搭配先前講過的例子，所以只是把例子裡面的做法放到 Function 裡面來。

須注意這樣的作法只是上課 Demo 用。

In [ ]:
results = []

for symbol in companylist:
    vo = volatility(symbol, datetime(2016, 1, 1))
    results.append((vo, symbol))
results.sort()

results

In [ ]:
# 選出波動率最小的前五檔股票
computer_selected = np.array(results)[:, 1][:5].tolist()
computer_selected

# 加入自選股

In [ ]:
# 上面是電腦幫忙選股...
# 底下是自己因為看新聞、聽小道消息等等的靈機一閃後，想看看看回測狀況好不好的股票
# 它只是一個簡單的 list 會被一起放進去回測看看
self_selected = ['TSLA', 'GOOG', 'YHOO', 'MSFT', 'AAPL']

In [ ]:
candidates = computer_selected + self_selected

In [ ]:
# 放到 set 中的重複性資料只會被保留一份
candidates = set(candidates)

In [ ]:
# 這是刪掉重複資料後的候選名單
candidates = list(candidates)

# 回測

In [ ]:
# 利用策略產生的持有部位資訊，計算底下四個指標來判斷投資績效
# sharpe ratio: 判斷報酬的好壞跟穩定度，數值越大越好
# maxdd: maximum drawdown, 最糟糕的狀況會賠幾 %
# maxddd: maximum drawdown duration, 低於上一次最高報酬的天數
# cumRet[-1]: 最後賺的 % 數

def indicators(df):
    dailyRet = df['Close'].pct_change()
    #假設無風險利率為 4%
    #假設一年有252個交易日
    excessRet = (dailyRet - 0.04/252)[df['positions']==1.0]
    SharpeRatio = np.sqrt(252.0)*np.mean(excessRet)/np.std(excessRet)
    
    cumRet = np.cumprod(1+excessRet)
    
    maxdd, maxddd = DrawDownAnalysis(cumRet)
    
    return SharpeRatio, maxdd, maxddd, cumRet[-1]

In [ ]:
# 這是我們的策略的部分
# 主要只是要算出進出的訊號 signals 跟何時持有部位 positions
# 底下是一個突破系統的範例

def breakout(df):
    # Donchian Channel
    df['20d_high'] = np.round(pd.Series.rolling(df['Close'], window=20).max(), 2)
    df['10d_low'] = np.round(pd.Series.rolling(df['Close'], window=10).min(), 2)

    has_position = False
    df['signals'] = np.zeros(df['Close'].shape)
    for t in range(2, df['signals'].size):
        if df['Close'][t] > df['20d_high'][t-1]:
            if not has_position:
                df.loc[df.index[t], 'signals'] = 1.0
                has_position = True
        elif df['Close'][t] < df['10d_low'][t-1]:
            if has_position:
                df.loc[df.index[t], 'signals'] = -1.0
                has_position = False

    df['positions'] = df['signals'].cumsum()
    return df

In [ ]:
def apply_strategy(strategy, df):
    return strategy(df)

In [ ]:
# 先把所有股票資料抓下來，放到字典上面去
# 因為我們在這裡沒有使用資料庫，所以用字典來做存放
all_data = {}

for symbol in candidates:
    all_data[symbol] = web.DataReader(symbol, 'google', datetime(2016,1,1))

In [ ]:
# 計算各支股票的回測結果
results = []

for symbol in candidates:
    apply_strategy(breakout, all_data[symbol])
    SharpeRatio, maxdd, maxddd, finalRet = indicators(all_data[symbol])
    results.append((SharpeRatio, maxdd, maxddd, finalRet, symbol))

results

In [ ]:
# 排序股票，取 Sharpe Ratio 高的前幾名當標的
sorted(results, reverse=True)

In [ ]:
results_df = pd.DataFrame(results, columns=['sharpe','MaxDrawDown','MaxDrawDownDuration','returns','symbols'])

In [ ]:
results_df

In [ ]:
results_df.sort_values('MaxDrawDown',ascending=False)